In [ ]:
import networkx as nx
from gnpsdata import taskresult

In [4]:
task = "cf6e14abf5604f47b28b467a513d3532"

In [68]:
# Downloading raw data from GNPS
def download_graphml(task, output_file):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", output_file)

def get_graphml_network(task):
    taskresult.download_task_resultfile(task, "gnps_molecular_network_graphml/", "temp.graphml")

    G = nx.read_graphml("temp.graphml")

    return G

def download_quantification(task, output_file):
    taskresult.download_task_resultfile(task, "quantification_table/", output_file)

def download_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "metadata_merged/", output_file)

def download_mgf(task, output_file):
    taskresult.download_task_resultfile(task, "spectra_reformatted/", output_file)
    
# Qiime2 Data
def download_qiime2(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_table.qza", output_file)

def download_qiime2_manifest(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_manifest.tsv", output_file)

def download_qiime2_metadata(task, output_file):
    taskresult.download_task_resultfile(task, "qiime2_output/qiime2_metadata.tsv", output_file)

In [85]:
# Download quantification and manifest
download_quantification(task, "../data/quant.csv")
download_qiime2_manifest(task, "../data/manifest.csv")

# Import Into Qiime2
## Convert .tsv to .biom
! source activate qiime2-2019.4 && biom convert -i ../data/gnps_quant/quant.tsv -o quant.biom --to-hdf5

In [80]:
import csv 

# Replace the following file names with your own 
mzmine_file = '../data/quant.csv' 
biom_file = '../data/biom_quantification.tsv' 

# Open the MZmine CSV file with 
with open(mzmine_file, 'r') as csv_file: 
    csv_reader = csv.reader(csv_file) 
    
    # Open the BIOM TSV file with 
    with open(biom_file, 'w', newline='') as tsv_file: 
        tsv_writer = csv.writer(tsv_file, delimiter='\t') 
        
        # Skip the first line of the MZmine CSV file 
        next(csv_reader) 
        
        # Write the header row of the BIOM TSV file 
        tsv_writer.writerow(['#OTU ID', 'sample_name', 'abundance']) 
        # Loop through the remaining lines of the MZmine CSV file
        for row in csv_reader: 
            # Extract the relevant columns from the MZmine CSV file 
            mzmine_id = row[0] 
            sample_name = row[1] 
            abundance = row[2] 
            # Write a row to the BIOM TSV file 
            tsv_writer.writerow([mzmine_id, sample_name, abundance]) 

In [81]:
! source activate qiime2-2019.4 && biom convert -i ../data/biom_quantification.tsv -o ../data/quant.biom --to-hdf5

In [83]:
! source activate qiime2-2019.4 && qiime tools import \
  --input-path ../data/quant.biom \
  --type 'FeatureTable[Frequency]' \
  --input-format BIOMV210Format \
  --output-path ../data/quant.qza

Imported ../data/quant.biom as BIOMV210Format to ../data/quant.qza


# ANOVA
! source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file <path-to-metadata-file.tsv> \
  --p-formula "independent_variable~dependent_variable+dependent_variable" \
  --o-visualization <path-to-visualization.qza>

In [106]:
! source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file ../data/metadata.tsv \
  --p-formula "ATTRIBUTE_Year~ATTRIBUTE_Sample_Area+ATTRIBUTE_Latitude" \
  --p-sstype 'I' \
  --o-visualization metadata.qzv

Saved Visualization to: metadata.qzv


# Visualization
Qiime2 visualizations do not work in headless environments, we can view them at https://view.qiime2.org/

In [62]:
# Qiime2 Manipulations Happen after this to mirror the other notebooks

! source activate qiime2-2019.4 && qiime

Usage: qiime [OPTIONS] COMMAND [ARGS]...

  QIIME 2 command-line interface (q2cli)
  --------------------------------------

  To get help with QIIME 2, visit https://qiime2.org.

  To enable tab completion in Bash, run the following command or add it to
  your .bashrc/.bash_profile:

      source tab-qiime

  To enable tab completion in ZSH, run the following commands or add them to
  your .zshrc:

      autoload bashcompinit && bashcompinit && source tab-qiime

Options:
  --version   Show the version and exit.
  --help      Show this message and exit.

Commands:
  info                Display information about current deployment.
  tools               Tools for working with QIIME 2 files.
  dev                 Utilities for developers and advanced users.
  alignment           Plugin for generating and manipulating alignments.
  composition         Plugin for compositional data analysis.
  cutadapt            Plugin for removing adapter sequences, primers, and
                      oth

# Principal Coordinate Analysis (PCoA) & Distance Matrix
! source activate qiime2-2019.4 && first create distance matrix 
 qiime diversity beta \
  --i-table <path-to-feature-table.qza> \
  --p-metric canberra_adkins \
  --output-dir <path-to-distance-matrix.qza> \

In [ ]:
! source activate qiime2-2019.4 && qiime diversity beta \
  --i-table qiime_table.qza \
  --p-metric canberra_adkins \
  --o-distance-matrix distance_matrix.qza

Usage: qiime diversity beta [OPTIONS]

  Computes a user-specified beta diversity metric for all pairs of samples
  in a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                       The feature table containing the samples over which
                       beta diversity should be computed.           [required]
Parameters:
  --p-metric TEXT Choices('yule', 'mahalanobis', 'chebyshev', 'kulsinski',
    'correlation', 'euclidean', 'canberra_adkins', 'cosine', 'aitchison',
    'jaccard', 'sokalmichener', 'hamming', 'seuclidean', 'sqeuclidean',
    'canberra', 'matching', 'cityblock', 'rogerstanimoto', 'sokalsneath',
    'braycurtis', 'wminkowski', 'dice', 'russellrao')
                       The beta diversity metric to be computed.    [required]
  --p-pseudocount INTEGER
    Range(1, None)     A pseudocount to handle zeros for compositional
                       metrics.  This is ignored for other metrics.
                                                   

## PCoA
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix <path-to-distance-matrix.qza> \
  --p-number-of-dimensions INTEGER \ #optional
  --o-pcoa <path-to-artifact.qza>

In [94]:
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix distance_matrix.qza \
  --o-pcoa pcoa.qza

Saved PCoAResults to: pcoa.qza


# Emperor plot
! source activate qiime2-2019.4 && qiime emperor plot \
--i-pcoa <path_to_pcoa.qza> \
--m-metadata-file <path_to_metadata.tsv \
--o-visualization emperor_qiime2

In [102]:
! source activate qiime2-2019.4 && qiime emperor plot \
  --i-pcoa ./pcoa.qza \
  --m-metadata-file ./metadata.tsv \
  --o-visualization emperor_plot

Saved Visualization to: emperor_plot.qzv
